## Import Library

In [0]:
# Install rdkit

def install_rdkit():
  !wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
  !chmod +x Miniconda3-latest-Linux-x86_64.sh
  !time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
  !time conda install -q -y -c conda-forge rdkit

# install_rdkit()

In [0]:
# Data

#from google.colab import files
import pandas as pd
import numpy as np
import regex as re

# rdkit
import sys
#sys.path.append('/usr/local/lib/python3.7/site-packages/')
from rdkit import Chem
from rdkit.Chem import DataStructs

# random
import random as rd

## Data

### Pre-Processing Data

In [0]:
def preprocessing_data(link_of_data) :
  dataset = pd.read_csv(link_of_data,sep=";")
  dataset = dataset.iloc[:,[0,7,9,10,11]]

  list_of_column_dataframe = dataset.columns
  list_of_dataset_after = []

  list_of_dataset_before = np.array(dataset)
  
  for data in list_of_dataset_before :
    if data[2] == "'='" :
      if data[4] == "nM" :
        if data[3] < 10000 :
          data[3] = data[3]/1000
          list_of_dataset_after.append(data)
  
  dataframe_of_dataset_after = pd.DataFrame(list_of_dataset_after)
  dataframe_of_dataset_after.columns = list_of_column_dataframe
  name_of_file_download = "Dataset_After.csv"
  dataframe_of_dataset_after.to_csv(name_of_file_download)
  files.download(name_of_file_download)
  return dataframe_of_dataset_after


In [0]:
#preprocessing_data("https://raw.githubusercontent.com/relfarizi/Final_Project/master/Data/Dataset_Before.csv")

### Data After Pre-Processing

In [0]:
# for colab

dataset = pd.read_csv('https://raw.githubusercontent.com/relfarizi/Final_Project/master/Data/Dataset_After.csv', index_col= 0)


#dataset = pd.read_csv('Data/Dataset_After.csv',index_col = 0)
dataset.head(5)

In [0]:
#dataset = dataset.drop_duplicates(keep=False,inplace=True)
dataset = dataset.dropna()
mol_smiles = dataset.iloc[:,1].values.tolist()
row_drop = []
for i in range(len(mol_smiles)) :
  #print(mol_smiles[i])
  if len(str(mol_smiles[i])) < 5 :
    row_drop.append(i)
print(row_drop)
dataset.drop(row_drop)
dataset.iloc[258,:]

## Probability Activity

### 1. Molecule Activity
Untuk mencari jumlah dari nilai aktivitas masing2 molekul

variabel yang digunakan :
* mol_name = list nama molekul
* mol_smile = list smile molekul
* mol_act = list berisi nilai IC50
* act = nilai aktivitas tiap molekul
* sum_act = Nilai jumlah dari a

In [0]:
mol_name = dataset.iloc[:,0].values.tolist()
mol_smiles = dataset.iloc[:,1].values.tolist()
mol_act = dataset.iloc[:,3].values.tolist()
sum_act = 0
for molekul in mol_act :
  act = np.exp(-molekul)
  sum_act  += act
print(sum_act)

### 2. Probability Succes


Mencari nilai probabilitas kesuksesan untuk setiap molekul

Variabel yang digunakan :
* n = banyaknya molekul
* mol_prob = list probabilitas setiap molekul
* act = nilai aktivitas molekul
* prob = nilai probabilitas kesuksesan molekul
* sum_pro = nilai jumlah dari Probabiliras Kesuksesan




In [0]:
n = len(mol_act)
#p_avg = 1/n
mol_prob = []

# act = probabilitas kesuksesan

sum_pro = 0
for molekul in mol_act :
  #act = (molekul[1]*p_avg*n)/sum_act
  act = np.exp(-molekul)
  prob = act/sum_act
  prob = round(prob,8)
  mol_prob.append(prob)
  sum_pro += prob
print(sum_pro)


## Cost (Trial)

Menentukan cost dari setiap molekul. cell ini baru coba di-generate hanya dengan mengalikan dengan sum_of_cost dengan probability kesuksesannya. sehingga nilai probabilitas terbesar diasumsikan memiliki cost yang besar pula

In [0]:
sum_of_cost = 500000
mol_cost = []
sum_cost = 0
for mol in mol_prob:
    cost = mol*sum_of_cost
    mol_cost.append(round(cost,4))
    sum_cost += cost
print(sum_cost)

## Fingerprint

berisi bit biner dari sifat yang menggambarkan suatu molekul (baca lagi)

Variabel yang digunakan :
* fingerprint suatu molekul (dalam bit biner)
* mol_fp = list dari fingerprint setiap molekul


In [0]:
mol_fp = []
i = 0
print(mol_smiles[258])
for molekul in mol_smiles :
  smiles = Chem.MolFromSmiles(str(molekul))
  fp = Chem.RDKFingerprint(smiles)
  #print(i)
  i+= 1
  mol_fp.append(fp)
print((mol_fp))

## To DataFrame

In [0]:
#df_mol = pd.DataFrame.from_dict(mol_act_dict, orient='index')
#df_mol.columns = ["IC50","Molecule Activity", "Probability Activity"]
#df_mol.sort_values(by=["IC50"], ascending=False)
#mol_name = pd.Series(mol_name)
#mol_act = pd.Series(mol_act)
#df_mol = pd.concat(mol_name,mol_act)
#print(len(mol_name))
#print(len(mol_smiles))
#print(len(mol_fp))
#print(len(mol_act))
#print(len(mol_prob))

df = pd.DataFrame({'mol_name':mol_name,
                       'mol_smiles' : mol_smiles,
                       'mol_act':mol_act,
                       'mol_fp' : mol_fp,
                       'mol_prob' : mol_prob,
                       'mol_cost' : mol_cost
                       })

df.head()

In [0]:
df_mol = df.iloc[:,[0,2,3,5]]
df_mol

## Individu
suatu fungsi yang bertujuan untuk me-return dari suatu populasi (kumpulan individu berdasarkan subset yang diberikan). 

return :
* mol_name
* rata-rata dari act
* rata-rata dari mol_fp

cara menyari diversity :

dengan menghitung semua similarity antar molekul kemudian hasilnya di rata2kan

sebagai contoh :

ada 3 (subset) molekul (A,B,C), maka menghitungnya

* A -> B
* A -> C
* B -> C

kemudian dirata2kan

In [0]:
n_subset = 5

def individu(subset):
    #subset = np.array(subset.values.tolist())
    mol_name = subset.iloc[:,0].values
    mol_act = subset.iloc[:,1].values
    mol_fp = subset.iloc[:,2].values
    #print(mol_name)
    #print(mol_act)
    #print(mol_fp[1])

    
    fp_list = []
    n = len(subset)
    
    for i in range(n):
        for j in range(n):
            if i > j :
                tmp_fp = DataStructs.FingerprintSimilarity(mol_fp[i],mol_fp[j])
                fp_list.append(tmp_fp)
                
    avg_act = np.mean(mol_act)
    avg_div = np.mean(fp_list)
    if [mol_name,avg_act,avg_div] is not None :
        return [mol_name,avg_act,avg_div]
    else :
        return []



### cek budget

mengecek apakah molekul yang terpilih (yang ada di subset) tidak melebihi budget yang diberikan

In [0]:
def is_budget_enough(subset,budget):
    subset =  subset.iloc[:,3].values
    sum_of_subset = sum(subset)
    if sum_of_subset <= budget :
        check = True
    else :
        check = False
    return [check,sum_of_subset]

### NSGA-II

#### Initialization Individu

Menginisialisasi individu yang akan digunakan berdasarkan subset yang ada

In [0]:
# Initialization of Individu
budget = 50
n_subset = 20

def initialization_individu(df_mol,budget,n_subset) :
    i = 0
    iteration = 100
    calon_individu = df_mol.sample(n_subset)

    while(not is_budget_enough(calon_individu,budget)[0]) and i < iteration :
        calon_individu = df_mol.sample(n_subset)
        i+=1
    if is_budget_enough(calon_individu,budget)[0]:
        return individu(calon_individu)
    else :
        return []

        
        

In [0]:
if initialization_individu(df_mol,budget,n_subset) != [] :
    print(initialization_individu(df_mol,budget,n_subset))
else :
    print("can't continue")

### Try Pymoo

In [0]:
from pymoo.algorithms.nsga2 import NSGA2
from pymoo.factory import get_problem, get_sampling, get_crossover, get_mutation
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter

problem = get_problem("zdt5")

print(problem)

algorithm = NSGA2(pop_size=100,
                  sampling=get_sampling("bin_random"),
                  crossover=get_crossover("bin_two_point"),
                  mutation=get_mutation("bin_bitflip"),
                  eliminate_duplicates=True)

res = minimize(problem,
               algorithm,
               ('n_gen', 500),
               seed=1,
               verbose=True)

print(res)


In [0]:
Scatter().add(res.F).show()

## For git (Can't be use yet)

file > save a copy in Github

In [0]:
#!git init

#!git config --global user.name “relfarizi”

#!git remote add origin https://github.com/relfarizi/Final_Project.git

#!git add .

#!git commit -m "Probability Activity"

#!git push

#!git status

### lampau

In [0]:
subset_row = []
sum_of_subset = 10

for i in range(sum_of_subset):
    random = rd.randint(0,len(df_mol))
    while random in subset_row:
        random = rd.randint(0,len(df_mol))
    subset_row.append(random)
    
print(subset_row)
calon_individu = df_mol.iloc[subset_row,:].values.tolist()
calon_individu

individu(calon_individu)

In [0]:
# Inisialisasi Individu

def initialization_individu() :
    for i in range(sum_of_subset):
        random = rd.randint(0,len(df_mol))
        while random in subset_row:
            random = rd.randint(0,len(df_mol))
        subset_row.append(random)
#subset_row = rd.sample(df_mol.values,10)
df
list_subset = []
for i in (df_mol).index :
    if i in subset_row:
        is_subset = 1
    else :
        is_subset = 0
    list_subset.append(is_subset)
print(list_subset)